In [ ]:
# Snowpark for Python
from snowflake.snowpark.types import DoubleType
import snowflake.snowpark.functions as F
#
from snowflake.ml.registry import registry

# RFM Customer Segmentation

In [ ]:

 create or replace view test.test.vRFM
         as
        (
with raw_data as
(
select
o_custkey ,
o_orderdate ,
o_totalprice 
from snowflake_sample_data.tpch_sf1.orders
)
,most_recent_purchase as (
select 
dateadd(day,1, max( o_orderdate )) dt
from raw_data
)
,data as (
select
o_custkey,
datediff(day, max(o_orderdate),most_recent_purchase.dt) Recency,
count(o_orderdate) Frequency,
sum(o_totalprice) Monetary,
sum(o_totalprice)/count(o_orderdate) Monetary_Avg
from raw_data
join most_recent_purchase
group by o_custkey, most_recent_purchase.dt
having datediff(day, max(o_orderdate),most_recent_purchase.dt)<=180
)
,segmented_data as (
select 
o_custkey,
Recency,
Frequency,
Monetary,
Monetary_Avg,
NTILE(5) OVER (ORDER BY Recency)  R,
NTILE(5) OVER (ORDER BY Frequency)  F,
NTILE(5) OVER (ORDER BY Monetary)  M,
R + F + M RFM,
case
    when RFM > 13 then 'Champion'
    when RFM > 10 then 'Loyal'
    when RFM > 6 then 'Potential'
    when RFM > 0 then 'Need Attention'
end RFM_segment
from data
)
select
o_custkey,
Recency::integer as Recency,
Frequency::integer as Frequency,
Monetary::numeric(38,5) as Monetary,
Monetary_Avg::numeric(38,5) as Monetary_Avg,
R::integer as R,
F::integer as F,
M::integer as M,
RFM::integer as RFM,
RFM_segment::varchar(20) as RFM_segment
from segmented_data
        );
      
  

In [ ]:
# Get Snowflake Session object
session = get_active_session()
session.sql_simplifier_enabled = True

# Add a query tag to the session.
session.query_tag = {"origin":"Segmentation", 
                     "name":"e2e_ml_snowparkpython", 
                     "version":{"major":1, "minor":0,},
                     "attributes":{"is_quickstart":1}}

In [ ]:
import pandas as pd
import numpy as np
#
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm as cm
from matplotlib.pyplot import figure
import seaborn as sns
#
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from scipy.stats import boxcox, anderson
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples, silhouette_score
#
from sklearn.decomposition import PCA
#
import warnings
warnings.filterwarnings("ignore")

In [ ]:
np.random.seed(42)

In [ ]:
def Chart3D (cat_column):



    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(projection='3d')


    for s in df[cat_column].unique():
        ax.scatter(df[df[cat_column]==s]['RECENCY'], df[df[cat_column]==s]['FREQUENCY'], df[df[cat_column]==s]['MONETARY'],  label=f'{s}')
    ax.set_xlabel('RECENCY')
    ax.set_ylabel('FREQUENCY')
    ax.set_zlabel('MONETARY')
    ax.legend(loc='upper right')

    plt.show()
#
def PieChart(cat_column):
    # Set Seaborn style
    sns.set(style="whitegrid")

    # Prepare data
    segment_counts = df[cat_column].value_counts()

    # Plot
    fig, ax = plt.subplots()
    ax.pie(segment_counts, labels=segment_counts.index, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    plt.title('Customer Segmentation')
    plt.show()
#
def SummaryStats(cat_column):
    
    summary_stats_df = pd.DataFrame()

    for segment in df[cat_column].unique():
        segment_data = df[df[cat_column] == segment]

        # Summary for numerical features
        num_summary = segment_data[numerical_features].agg(['mean', 'std', 'min', 'max', 'count']).unstack()

        # Adding the segment summary to the overall summary
        summary_stats_df = pd.concat([summary_stats_df, num_summary], axis=1)

    # Naming the columns after the segments
    summary_stats_df.columns = [f"Segment_{seg}" for seg in df[cat_column].unique()]
    
    return summary_stats_df

def BoxPlots(cat_column):
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(20, 15))

    # Iterate through categorical columns and plot bar charts
    for j, cn in enumerate(numerical_features):
        for i, cc in enumerate([cat_column]):
            row = j
            col = i    
            sns.boxplot(x = cc, y =  cn, data=df, showfliers=False, showcaps=True, whiskerprops={'linewidth': 1},  ax=axes[row])
            axes[row].set_title(f'{cn}: {cc}')
    plt.subplots_adjust(hspace=2)
    plt.tight_layout()
    plt.show()

In [ ]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file
# We can now infer schema from CSV files.
df = session.sql("""
    


select
o_custkey ,
RECENCY,
FREQUENCY,
MONETARY ,
RFM_Segment
from test.test.vRFM
    
    """).to_pandas()

df.head()

In [ ]:
len(df)

In [ ]:
unique_id=['O_CUSTKEY']
numerical_features=['RECENCY', 'FREQUENCY', 'MONETARY']

## RFM Segments

In [ ]:
df['RFM_SEGMENT'].unique()

In [ ]:
PieChart('RFM_SEGMENT')

In [ ]:
Chart3D ('RFM_SEGMENT')

In [ ]:
BoxPlots('RFM_SEGMENT')

In [ ]:
SummaryStats('RFM_SEGMENT')

## Data Exploration

In [ ]:
columns = df.shape[1]

# Check normality assumption for numerical values (no dummies)
for c in df[numerical_features].columns:
    data = df[c]

    # Visual Inspection: Histogram and Q-Q plot
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    sns.histplot(data, kde=True)
    plt.title(f'Histogram of {c}')

    plt.subplot(1, 2, 2)
    stats.probplot(data, dist="norm", plot=plt)
    plt.title(f'Q-Q plot of {c}')

    plt.tight_layout()
    plt.show()

In [ ]:
# Create subplots with 1 column
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(20, 15))


for j, cn in enumerate(numerical_features):
    row = j
    sns.boxplot(y =  cn, data=df,  ax=axes[row])
    axes[row].set_title(f'{cn}')
plt.subplots_adjust(hspace=2)
plt.tight_layout()
plt.show()

With the exception of Recency, which is really a more discrete variable, Frequency and Monetary seems to have been normalized quite nicely. Well, allow some 'slack' here.

# KMeans Clustering

In [ ]:
#Scaling
zcols = ['z_'+c for c in numerical_features]

scaler=StandardScaler()
scaled_data = scaler.fit_transform(df[numerical_features])
scaled_df = pd.DataFrame(scaled_data, index=df.index, columns=zcols)

In [ ]:
def find_optimal_k(df, max_k):
    """
    Apply the elbow method to find the optimal number of clusters (k) for KMeans clustering.

    Parameters:
    df (DataFrame): The dataframe containing the data to cluster.
    max_k (int): The maximum number of clusters to try.

    Returns:
    None: This function plots a graph showing the elbow curve.
    """
    ssd = []  # Sum of squared distances

    for k in range(1, max_k + 1):
        kmeans = KMeans(n_clusters=k, n_init=42, random_state=42)
        kmeans.fit(df)
        ssd.append(kmeans.inertia_)

    plt.figure(figsize=(10, 6))
    plt.plot(range(1, max_k + 1), ssd, marker='o')
    plt.title('Elbow Method for Optimal K')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Sum of Squared Distances')
    plt.xticks(range(1, max_k + 1))
    plt.grid(True)
    plt.show()

find_optimal_k(scaled_df[zcols], 10)

In [ ]:
##########################################################################
## CREDIT:
##    This code has been adapted from
##    https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
##########################################################################
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9]
s_avg = []

for n_clusters in range_n_clusters:
    # Data to train the cluster model
    X = scaled_df.to_numpy()

    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, init='k-means++', n_init=42, random_state=42)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    s_avg.append(silhouette_avg)
    print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    #ax2.scatter(space_2d[..., 0], space_2d[..., 1], marker='.', s=30, lw=0, alpha=0.7, c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o', c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on RFM data "
                  "with n_clusters = %d" % n_clusters), fontsize=14, fontweight='bold')

plt.show()

In [ ]:
# Let's build the model with 4 components
kmeans = KMeans(n_clusters=4, init='random', max_iter=100, n_init=42, random_state = 42).fit(scaled_df)

# Assigning the segments to the original DataFrame
df['KMEANS_SEGMENT'] = kmeans.predict(scaled_df)

metric=silhouette_score(scaled_df, df['KMEANS_SEGMENT'])

In [ ]:
PieChart('KMEANS_SEGMENT')

In [ ]:
Chart3D ('KMEANS_SEGMENT')

In [ ]:
BoxPlots('KMEANS_SEGMENT')

In [ ]:
SummaryStats('KMEANS_SEGMENT')

- Segment 2 (red) - Recent, frequent and spent more money
- Segment 0 (green) - Purchace long time ago , frequent and spent more money
- Segment 3 (blue) - Recent, rare and less money
- Segment 1 (orange) - Purchace long time ago , rare and spent less money

In [ ]:
# database and schema
db = "CONTROL_DB"
schema = "MODELS"

# Get sample input data to pass into the registry logging function
sample_data = scaled_df.head(100)

# model name
model_name = "KMeans_Customer_Segmentation"

# create a registry and log the model
model_registry = registry.Registry(session=session, database_name=db, schema_name=schema)

# log the fitted model
model_ver = model_registry.log_model(
    model_name=model_name,
    model=kmeans,
    sample_input_data=sample_data
)

# evaluation metric
model_ver.set_metric(metric_name="silhouette_score", value=metric )

# comments
model_ver.comment = "This is KMeans model to segment customers based on Recency, Frequency and Monetary"

# check
model_registry.get_model(model_name).show_versions()

# Gaussian Segmentation

This clustering method is sensitive to normal distribution. 
Checking normality and transforming if needed

In [ ]:
class BoxCoxTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, alpha=0.05):
        self.alpha = alpha

    def fit(self, X, y=None):
        self.lambdas_ = {}
        self.shifts_ = {}
        for col in X.columns:
            # Shift data if necessary
            min_val = X[col].min()
            shift = 0 if min_val > 0 else -min_val + 1
            self.shifts_[col] = shift
            shifted_data = X[col] + shift

            # Apply Anderson-Darling test
            ad_test_result = anderson(shifted_data.dropna())
            if ad_test_result.statistic > self.alpha: #ad_test_result.critical_values[0]:  # Comparing with the critical value at 5% significance level
                _, maxlog = boxcox(shifted_data.dropna())
                self.lambdas_[col] = maxlog
        return self

    def transform(self, X):
        X_transformed = pd.DataFrame(index=X.index)
        for col in X.columns:
            shifted_data = X[col] + self.shifts_.get(col, 0)
            if col in self.lambdas_:
                transformed_col = boxcox(shifted_data, lmbda=self.lambdas_[col])
                X_transformed[col] = transformed_col
            else:
                X_transformed[col] = shifted_data
        return X_transformed

In [ ]:
# Create a pipeline for numerical features
numerical_pipeline = Pipeline([
    ('boxcox', BoxCoxTransformer()),
    ('scaler', StandardScaler())
])

In [ ]:
# Create a column transformer to combine the pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_pipeline, numerical_features)
    ],
    remainder='passthrough'
)

In [ ]:
transformed_data = preprocessor.fit_transform(df[numerical_features])

In [ ]:
# Create the final DataFrame
transformed_df = pd.DataFrame(transformed_data,index = df.index)

In [ ]:
def find_best_gmm(X, k_range=(2, 8)):
    best_aic = np.inf
    best_bic = np.inf
    best_k = None
    best_gmm = None
    aic_values = []
    bic_values = []

    for k in k_range:
        gmm = GaussianMixture(n_components=k, random_state=0).fit(X)
        aic = gmm.aic(X)
        bic = gmm.bic(X)

        # Store the various AICs and BICs
        aic_values.append(aic)
        bic_values.append(bic)

        if aic < best_aic:
            best_aic = aic
            best_bic = bic
            best_k = k
            best_gmm = gmm

    return best_gmm, best_k, best_aic, best_bic, aic_values, bic_values

# Example usage
# Assuming 'transformed_df' is the DataFrame from the pipeline
k_range = range(3,12)
best_gmm_model, best_k, best_aic, best_bic, aics, bics = find_best_gmm(transformed_df, k_range)
print(f"Best GMM Model: {best_k} segments")
print(f"AIC: {best_aic}")
print(f"BIC: {best_bic}")

In [ ]:
# Plotting AIC and BIC
plt.plot ([k for k in k_range], aics, label='AIC')
plt.plot ([k for k in k_range], bics, label='BIC')
plt.legend()
plt.xlabel('Number of Segments')

While the AIC and BIC continue to decrease as we increase the components, we are looking to the simplest (less complex) model - 4 segments.

In [ ]:
# Let's build the model with 4 components
gmm = GaussianMixture(n_components=4, random_state=42).fit(transformed_df)

# Assigning the segments to the original DataFrame
df['GMM_SEGMENT'] = gmm.predict(transformed_df)

metric = gmm.aic(transformed_df)

In [ ]:
PieChart('GMM_SEGMENT')

In [ ]:
Chart3D ('GMM_SEGMENT')

In [ ]:
BoxPlots('GMM_SEGMENT')

In [ ]:
SummaryStats('GMM_SEGMENT')

- Segment 1 (green) - Recent, frequent and spent more money
- Segment 3 (red) - Purchace long time ago , frequent and spent more money
- Segment 0 (blue) - Recent, rare and less money
- Segment 2 (orange) - Purchace long time ago , rare and spent less money

In [ ]:
# database and schema
db = "CONTROL_DB"
schema = "MODELS"

# Get sample input data to pass into the registry logging function
sample_data = scaled_df.head(100)

# model name
model_name = "GMM_Customer_Segmentation"

# create a registry and log the model
model_registry = registry.Registry(session=session, database_name=db, schema_name=schema)

# log the fitted model
model_ver = model_registry.log_model(
    model_name=model_name,
    #version_name='V0',
    model=kmeans,
    sample_input_data=sample_data
)

# evaluation metric
model_ver.set_metric(metric_name="AIC", value=metric )

# comments
model_ver.comment = "This is GMM model to segment customers based on Recency, Frequency and Monetary"

# check
model_registry.get_model(model_name).show_versions()